# BioExplorer - CCFv3
![](../bioexplorer_ccfv3_banner.png)

In [ ]:
import numpy as np
import json

def compute_transform_matrix(position, forward, up):
    """
    Compute the 4x4 transformation matrix for a camera given its position, forward, and up vectors.
    
    :param position: A 3D vector [x, y, z] representing the camera position.
    :param forward: A 3D vector representing the forward direction of the camera.
    :param up: A 3D vector representing the up direction of the camera.
    :return: A 4x4 transformation matrix.
    """
    # Normalize the forward vector
    forward = forward / np.linalg.norm(forward)
    
    # Compute the right vector (cross product of up and forward)
    right = np.cross(up, forward)
    right = right / np.linalg.norm(right)
    
    # Recompute the up vector to ensure orthogonality
    up = np.cross(forward, right)
    
    # Build the rotation matrix
    rotation_matrix = np.column_stack((right, up, -forward))
    
    # Append the position as the last column of the 4x4 matrix
    transform_matrix = np.eye(4)
    transform_matrix[:3, :3] = rotation_matrix
    transform_matrix[:3, 3] = position
    
    return transform_matrix.tolist()  # Return as a list for JSON serialization

def generate_transforms_json(positions, directions, up_vectors, image_paths, fov_x, output_file, img_width, img_height):
    """
    Generate a transforms.json file using camera positions, directions, up vectors, and image paths.
    
    :param positions: List of 3D camera positions.
    :param directions: List of forward direction vectors.
    :param up_vectors: List of up vectors.
    :param image_paths: List of corresponding image paths.
    :param fov_x: The horizontal field of view (in radians).
    :param output_file: Path to save the transforms.json file.
    :param img_width: Width of the image.
    :param img_height: Height of the image.
    """
    # Calculate fx from the field of view and image width
    fx = 0.5 * img_width / np.tan(0.5 * fov_x)
    fy = fx  # Assuming square pixels
    cx = img_width / 2.0
    cy = img_height / 2.0
    
    data = {
        "fl_x": fx,
        "fl_y": fy,
        "k1": -0.013472197525381842,
        "k2": 0.007509466554079491,
        "p1": -0.0011800209664517077,
        "p2": 0.01116939407701522,
        "cx": cx,
        "cy": cy,
        "w": 3840,
        "h": 2160,
        "aabb_scale": 16,
        "frames": []
    }
    
    # Generate transformation matrices and intrinsics for each frame
    for position, forward, up, image_path in zip(positions, directions, up_vectors, image_paths):
        transform_matrix = compute_transform_matrix(position, forward, up)
        frame_data = {
            "file_path": image_path,
            "transform_matrix": transform_matrix,
        }
        data["frames"].append(frame_data)
    
    # Save to JSON
    with open(output_file, 'w') as f:
        json.dump(data, f, indent=4)


In [ ]:
from bioexplorer import BioExplorer, MovieMaker
import os

be=BioExplorer('localhost:5000')
core = be.core_api()
mm = MovieMaker(be)

In [ ]:
import math
from tqdm import tqdm
import os

image_folder = '/scratch/ccfv3a/orbital/v2'

k = 4

params = core.OrthographicCameraParams()
params.height = 12000
status = core.set_camera_params(params)

fov_x = 60.0
positions = list()
directions = list()
up_vectors = list()
image_paths = list()

img_width = 1920
img_height = 1080

r = params.height
t = [7062, 3849, 5687]
frame = 0
for i in tqdm(range(270, -90, -1)):
    o = [
        t[0] + r * math.cos(i * math.pi / 180.0),
        t[1],
        t[2] + r * math.sin(i * math.pi / 180.0)
    ]
    l = 0.0
    d = [0,0,0]
    for k in range(3):
        d[k] = t[k] - o[k]
        l += d[k] * d[k]

    l = math.sqrt(l)
    for k in range(3):
        d[k] /= l

    positions.append(o)
    directions.append(d)
    up_vectors.append([0,-1,0])
    image_paths.append('./%05d.png' % frame)
    frame += 1

generate_transforms_json(
    positions, directions, up_vectors, image_paths,
    fov_x, os.path.join(image_folder, 'transforms.json'), 
    img_width, img_height)
